In [1]:
# Imports and installations
!pip install igraph

from Ring import Node, Direction, Ring
from Algorithms import MinMax, MinMaxPlus
import random
import time
import math
import matplotlib.pyplot as plt

In [2]:
# Defining the functions to run the program
def generate_random_ring(size):
    temp = [i for i in range(1, size + 1)]
    random.shuffle(temp)
    nodes = [Node(value, None, None) for value in temp]

    return nodes


def running_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    elapsed_milliseconds = round((end_time-start_time) * 1000)
    return elapsed_mins, elapsed_secs, elapsed_milliseconds


def outer_experimental_loop():
    percent_of_originators = [0.1, 0.2, 0.5, 0.75, 1]
    sizes_of_ring = [10, 20, 30, 40, 50, 75]

    ################# First Experiment #################
    results_min_max = []
    results_min_max_plus = []

    # Effect of changing the ring size while holding number of originators constant
    for ring_size in sizes_of_ring:
        print(ring_size)
        messages_min_max, messages_min_max_plus = run_experiments(number_of_originators=round(ring_size * percent_of_originators[0]), size_of_ring=ring_size, animation=False, printing=False)

        # Append results
        results_min_max.append(messages_min_max)
        results_min_max_plus.append(messages_min_max_plus)

    # Plot the data
    plt.plot(sizes_of_ring, results_min_max, label='min_max')
    plt.plot(sizes_of_ring, results_min_max_plus, label='min_max_plus')

    # Add a legend
    plt.legend()

    # Add labels
    plt.xlabel("Ring size")
    plt.ylabel("Number of messages")
    plt.title("Min_Max vs. Min_Max_Plus with percent_of_originators=10%")

    # Show the plot
    plt.show()
    ################# First Experiment #################

    ################# Second Experiment #################
    results_min_max = []
    results_min_max_plus = []

    # Effect of changing the number of originators while holding the ring size constant
    for percent in percent_of_originators:
        messages_min_max, messages_min_max_plus = run_experiments(number_of_originators=round(sizes_of_ring[1] * percent), size_of_ring=sizes_of_ring[1], animation=False, printing=False)

        # Append results
        results_min_max.append(messages_min_max)
        results_min_max_plus.append(messages_min_max_plus)

    # Plot the data
    plt.plot(percent_of_originators, results_min_max, label='min_max')
    plt.plot(percent_of_originators, results_min_max_plus, label='min_max_plus')

    # Add a legend
    plt.legend()

    # Add labels
    plt.xlabel("Percent of originators")
    plt.ylabel("Number of messages")
    plt.title("Min_Max vs. Min_Max_Plus with ring_size=20")

    # Show the plot
    plt.show()
    ################# Second Experiment #################


def run_experiments(number_of_originators=2, size_of_ring=10, direction=Direction.RIGHT, animation_speed=500, animation=True, printing=True):
    # Let's generate a couple nodes to start and make sure we can graph them properly
    nodes_min_max = generate_random_ring(size_of_ring)
    nodes_min_max_plus = [Node(node.value, None, None) for node in nodes_min_max]

    # Get both algorithms
    min_max = MinMax()
    min_max_plus = MinMaxPlus()

    # Now let's link up the nodes in a ring
    ring_min_max = Ring(nodes_min_max, direction, min_max, number_of_originators)
    ring_min_max_plus = Ring(nodes_min_max_plus, direction, min_max_plus, number_of_originators)

    # Now let's test out leader election in the ring
    # MinMax
    start_time = time.time()
    leader_node_min_max, messages_min_max = \
        ring_min_max.leader_election()
    end_time = time.time()
    mins_min_max, secs_min_max, mils_min_max = running_time(start_time, end_time)


    # MinMaxPlus
    start_time = time.time()
    leader_node_min_max_plus, messages_min_max_plus = \
        ring_min_max_plus.leader_election()
    end_time = time.time()
    mins_min_max_plus, secs_min_max_plus, mils_min_max_plus = running_time(start_time, end_time)

    if printing:
        # Print all edges in order
        print(f"The edges (in direction {direction.value}) for the ring executing min-max are: "
              f"{[elem.get_edge(Direction.RIGHT) for elem in ring_min_max.nodes]}")
        print(f"The edges (in direction {direction.value}) for the ring executing min-max-plus are: "
              f"{[elem.get_edge(Direction.RIGHT) for elem in ring_min_max_plus.nodes]}")

        # Print min_max
        print(f"\n\nWe have elected a leader for min-max: {leader_node_min_max}.")
        print(f'Running Time: {mins_min_max}m {secs_min_max}s. Only in milliseconds: {mils_min_max}ms.')
        print(f'Theoretical message upper bound: {math.ceil(1.44 * size_of_ring * math.log(size_of_ring, 2))}.')
        print(f"It required a total of {messages_min_max} messages\n\n.")

        # Print min_max_plus
        print(f"We have elected a leader for min-max-plus: {leader_node_min_max_plus}.")
        print(f'Running Time: {mins_min_max_plus}m {secs_min_max_plus}s. Only in milliseconds: {mils_min_max_plus}ms.')
        print(f'Theoretical message upper bound: {math.ceil(1.271 * size_of_ring * math.log(size_of_ring, 2))}.')
        print(f"It required a total of {messages_min_max_plus} messages.")

    # Time to visualize the graph
    if animation:
        ring_min_max.visualize(animation_speed=animation_speed)
        ring_min_max_plus.visualize(animation_speed=animation_speed)

    return messages_min_max, messages_min_max_plus

In [4]:
# Run the program


# number_of_originator defines how many originators there are in the algorithm.
# This value cannot be greater than the size of the ring.

# size_of_the_ring defines the size of the ring.

# direction is either Direction.LEFT or Direction.RIGHT.

# animation_speed is initially set at 500. This is the number of milliseconds per frame. Increase this value
# to make the animation longer for each frame.

run_experiments(number_of_originators=5, size_of_ring=10, direction=Direction.RIGHT, animation_speed=500)

# outer_experimental_loop()

The edges (in direction Right) for the ring executing min-max are: [[2, 3], [3, 5], [5, 4], [4, 3], [3, 9], [9, 8], [8, 6], [6, 3], [3, 1], [1, 2]]
The edges (in direction Right) for the ring executing min-max-plus are: [[2, 7], [7, 4], [4, 7], [7, 3], [3, 9], [9, 4], [4, 6], [6, 4], [4, 1], [1, 2]]


We have elected a leader for min-max: 3.
Running Time: 0m 0s. Only in milliseconds: 48ms.
Theoretical message upper bound: 48.
It required a total of 30 messages

.
We have elected a leader for min-max-plus: 4.
Running Time: 0m 0s. Only in milliseconds: 49ms.
Theoretical message upper bound: 43.
It required a total of 33 messages.


IndexError: list index out of range

AttributeError: 'VertexCollection' object has no property 'offset_transform'

<Figure size 432x288 with 1 Axes>